# Main imports and code

In [7]:
!zip task4-hw4.zip /content/drive/MyDrive/task4-hw4/* 

  adding: content/drive/MyDrive/task4-hw4/sushma/ (stored 0%)
  adding: content/drive/MyDrive/task4-hw4/task4-final-project-assignment.gdoc
zip warning: Operation not supported
	zip warning: could not open for reading: content/drive/MyDrive/task4-hw4/task4-final-project-assignment.gdoc
  adding: content/drive/MyDrive/task4-hw4/waad/ (stored 0%)

zip warning: Not all files were readable
  files/entries read:  2 (0 bytes)  skipped:  1 (151 bytes)


In [ ]:
# check which gpu we're using
!nvidia-smi

Wed Dec 15 05:33:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/
!cp /content/drive/MyDrive/nlp-final-project/dontpatronizeme_categories.tsv ./
!cp /content/drive/MyDrive/nlp-final-project/dontpatronizeme_pcl.tsv ./
!cp /content/drive/MyDrive/nlp-final-project/README.txt ./

/content


In [ ]:
!pip install simpletransformers
!pip install tensorboardx

     |████████████████████████████████| 247 kB 14.9 MB/s 
     |████████████████████████████████| 3.3 MB 36.9 MB/s 
     |████████████████████████████████| 9.1 MB 53.4 MB/s 
     |████████████████████████████████| 3.3 MB 51.0 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 1.7 MB 47.5 MB/s 
     |████████████████████████████████| 1.2 MB 46.0 MB/s 
     |████████████████████████████████| 298 kB 64.0 MB/s 
     |████████████████████████████████| 61 kB 603 kB/s 
     |████████████████████████████████| 596 kB 57.7 MB/s 
     |████████████████████████████████| 895 kB 66.9 MB/s 
     |████████████████████████████████| 140 kB 74.6 MB/s 
     |████████████████████████████████| 180 kB 68.1 MB/s 
     |████████████████████████████████| 97 kB 8.9 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 132 kB 65.6 MB/s 
     |████████████████████████████████| 1.1 MB 64.3 MB/s 
     |████████████████

     |████████████████████████████████| 124 kB 13.2 MB/s 


In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
from urllib import request
import pandas as pd
import logging
import torch
from collections import Counter
from ast import literal_eval

In [ ]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

Cuda available?  True


In [ ]:
if cuda_available:
  import tensorflow as tf
  # Get the GPU device name.
  device_name = tf.test.gpu_device_name()
  # The device name should look like the following:
  if device_name == '/device:GPU:0':
      print('Found GPU at: {}'.format(device_name))
  else:
      raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


# Fetch Don't Patronize Me! data manager module

In [ ]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [ ]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [ ]:
from dont_patronize_me import DontPatronizeMe

In [ ]:
dpm = DontPatronizeMe('.', '.')

In [ ]:
dpm.load_task1()
dpm.load_task2(return_one_hot=True)

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


In [ ]:
! git clone https://github.com/Perez-AlmendrosC/dontpatronizeme.git

Cloning into 'dontpatronizeme'...
remote: Enumerating objects: 200, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 200 (delta 87), reused 60 (delta 21), pack-reused 0
Receiving objects: 100% (200/200), 1.64 MiB | 4.67 MiB/s, done.
Resolving deltas: 100% (87/87), done.


# Load paragraph IDs

In [ ]:
trids = pd.read_csv('/content/dontpatronizeme/semeval-2022/practice splits/train_semeval_parids-labels.csv')
teids = pd.read_csv('/content/dontpatronizeme/semeval-2022/practice splits/dev_semeval_parids-labels.csv')

In [ ]:
trids.head()

,par_id,label
0,4341,"[1, 0, 0, 1, 0, 0, 0]"
1,4136,"[0, 1, 0, 0, 0, 0, 0]"
2,10352,"[1, 0, 0, 0, 0, 1, 0]"
3,8279,"[0, 0, 0, 1, 0, 0, 0]"
4,1164,"[1, 0, 0, 1, 1, 1, 0]"


In [ ]:
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

# Rebuild training set (Task 1)

In [ ]:
rows = [] # will contain par_id, label and text
for idx in range(len(trids)):  
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'text':text,
      'label':label
  })
  

In [ ]:
trdf1 = pd.DataFrame(rows)

# Rebuild test set (Task 1)

In [ ]:
rows = [] # will contain par_id, label and text
for idx in range(len(teids)):  
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'text':text,
      'label':label
  })
  

In [ ]:
len(rows)

2094

In [ ]:
tedf1 = pd.DataFrame(rows)

# SpanBERT Baseline for Task 1

In [ ]:
# downsample negative instances
pcldf = trdf1[trdf1.label==1]
npos = len(pcldf)

training_set1 = pd.concat([pcldf,trdf1[trdf1.label==0][:npos*2]])

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [ ]:
training_set1

,par_id,text,label
0,4341,"the scheme saw an estimated 150,000 children f...",1
1,4136,durban 's homeless communities reconciliation ...,1
2,10352,the next immediate problem that cropped up was...,1
3,8279,far more important than the implications for t...,1
4,1164,to strengthen child-sensitive social protectio...,1
...,...,...,...
2377,1775,last but not the least element of culpability ...,0
2378,1776,"then , taking the art of counter-intuitive non...",0
2379,1777,kagunga village was reported to lack necessary...,0
2380,1778,"""after her parents high-profile divorce after ...",0


In [ ]:

task1_model_args = ClassificationArgs(num_train_epochs=1, 
                                      no_save=True, 
                                      no_cache=True, 
                                      overwrite_output_dir=True)
task1_model = ClassificationModel("bert", 
                                  'mrm8488/spanbert-base-finetuned-tacred', 
                                  args = task1_model_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)
# train model
task1_model.train_model(training_set1[['text', 'label']])
# run predictions
preds_task1, _ = task1_model.predict(tedf1.text.tolist())

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mrm8488/spanbert-base-finetuned-tacred and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/298 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2094 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

In [ ]:
!pip install keybert

     |████████████████████████████████| 78 kB 6.1 MB/s 
     |████████████████████████████████| 214 kB 27.0 MB/s 
     |████████████████████████████████| 51 kB 8.6 MB/s 
  Created wheel for keybert: filename=keybert-0.5.0-py3-none-any.whl size=20491 sha256=cdf50333eae04a1706e39b01b7a644e2c12b9e840b82a710e5194c4215590db2
  Stored in directory: /root/.cache/pip/wheels/99/1f/3f/590d2997adbb2d0e1f82e8ee05d42d6910e92c3ed283015ff8
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=4852ed7f2bfe657ccd7103c10e776022fd86485caebc7864f476e60c7c6e3744
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built keybert sentence-transformers


In [ ]:
!pip install keybert[spacy]

In [ ]:
from keybert import KeyBERT

doc = """
         Supervised learning is the machine learning task of learning a function that
         maps an input to an output based on example input-output pairs. It infers a
         function from labeled training data consisting of a set of training examples.
         In supervised learning, each example is a pair consisting of an input object
         (typically a vector) and a desired output value (also called the supervisory signal). 
         A supervised learning algorithm analyzes the training data and produces an inferred function, 
         which can be used for mapping new examples. An optimal scenario will allow for the 
         algorithm to correctly determine the class labels for unseen instances. This requires 
         the learning algorithm to generalize from the training data to unseen situations in a 
         'reasonable' way (see inductive bias).
      """
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(doc)


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


In [ ]:
kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 6), stop_words=None)

[('supervised learning is the machine learning', 0.7344),
 ('supervised learning is', 0.72),
 ('called the supervisory signal supervised learning', 0.7125),
 ('supervised learning algorithm analyzes the training', 0.7094),
 ('supervised learning is the', 0.7072)]

In [ ]:
keywords =[]
for sent in tedf1.text.tolist()[:20]:
  keywords.append({ sent: kw_model.extract_keywords(sent, keyphrase_ngram_range=(1, 12), stop_words=None,highlight=True )})

we also know that they can benefit by receiving counseling from someone who help them 
understand that their feelings are normal and that their situation is not hopeless ; someone 
who can help them put their situation in perspective and help them communicate with others 
who could provide support ; someone knowledgeable about resources they can access ; someone 
who can help them plan for their needs and the needs of their child by developing either a 
parenting plan or an adoption plan .

pope francis washed and kissed the feet of muslim , orthodox , orthodox , hindu and catholic 
refugees in a moving ceremony during holy thursday mass at a refugee centre on the outskirts 
of rome . ( march 24 )

many refugees do n't want to be resettled anywhere , let alone in the us .

"budding chefs , like "" fred "" , "" winston "" and "" angela "" in the kitchen are either 
homeless or recently re-housed - and desperately in need of a helping hand to enable them to 
re-build their shattered lives ."

"in a 90-degree view of his constituency , one can see a high rise and a flyover while 
underneath it , homeless people sleep on pavements . "" as you can see , my constituency is 
not confined to one socio-economic group , "" he said , adding that apart from water , 
another major concern for him is to improve the standard of public schools in the area ."

...

real poverty of britain : shocking images of uk in the sixties where poor really meant poor 
these hard-hitting photographs offer a glimpse into the harrowing day-to-day for poor 
families living in britain during the sixties .

valerie siddiq : my heart goes out to all the deceased and the poor families . may god grant 
us tolerance for all race and religions .

"our country is in need of serious change . we can continue to celebrate women 's day , throw
massive budgets at events , give out goody bags with a cute lipsticks and celebrate female 
achievers once a year . this is without taking away from exceptional women who truly are 
making inroads into their environments but we are not just a cute lipstick , or pair of shoes
, we are not just pretty and soft and squishy . we are not just child bearers , and there for
procreation . so , why are we as women , so accepting of the placating and mollification 
thrown at us and overlook the inequalities through acceptance . i am by no means militant , 
but often , as even in politics , focus is shifted on true issues by drawing public attention
to other "" things "" in order for us to be distracted . by shifting our focus from relevant 
issues we forget why we were disgruntled and angry and nothing ever changes ."

the world health organization did not give a reason for the increase in deaths , but a 
provincial health official in sindh said that the disease hit areas where poor families did 
not their children .

a top health official said today that the government could consider subsidies to help poor 
families pay for healthy food -- or imposing taxes on unhealthy products -- if other efforts 
fail to encourage better eating habits among hong kong . dr regina ching from the health 
department said such moves could be explored as a way to cut levels of chronic illnesses in 
the city , such ...

In [ ]:
keywords = dpm.train_task1_df['keyword'].unique().tolist()
keywords

['hopeless',
 'migrant',
 'immigrant',
 'disabled',
 'refugee',
 'in-need',
 'homeless',
 'vulnerable',
 'women',
 'poor-families']

In [ ]:
all_keywords =[]
for sent in tedf1.text.tolist():
  all_keywords.append({ sent: kw_model.extract_keywords(sent, candidates=keywords, keyphrase_ngram_range=(4, 10), stop_words=None)})

In [ ]:
all_keywords

[{'we also know that they can benefit by receiving counseling from someone who can help them understand that their feelings are normal and that their situation is not hopeless ; someone who can help them put their situation in perspective and help them communicate with others who could provide support ; someone knowledgeable about resources they can access ; someone who can help them plan for their needs and the needs of their child by developing either a parenting plan or an adoption plan .': [('poor-families',
    0.3067),
   ('homeless', 0.1921),
   ('refugee', 0.1698),
   ('in-need', 0.1157),
   ('disabled', 0.1155)]},
 {'pope francis washed and kissed the feet of muslim , orthodox , hindu and catholic refugees in a moving ceremony during holy thursday mass at a refugee centre on the outskirts of rome . ( march 24 )': [('refugee',
    0.3096),
   ('migrant', 0.2495),
   ('immigrant', 0.1789),
   ('homeless', 0.1779),
   ('poor-families', 0.1278)]},
 {"many refugees do n't want to b

In [ ]:
all_phrases =[]
for sent in tedf1.text.tolist()[:20]:
  all_phrases.append({ sent: kw_model.extract_keywords(sent, keyphrase_ngram_range=(1, 10), stop_words=None)})

In [ ]:
all_phrases

[{'we also know that they can benefit by receiving counseling from someone who can help them understand that their feelings are normal and that their situation is not hopeless ; someone who can help them put their situation in perspective and help them communicate with others who could provide support ; someone knowledgeable about resources they can access ; someone who can help them plan for their needs and the needs of their child by developing either a parenting plan or an adoption plan .': [('they can benefit by receiving counseling from someone who can',
    0.742),
   ('they can benefit by receiving counseling from someone', 0.7409),
   ('they can benefit by receiving counseling from someone who', 0.7404),
   ('benefit by receiving counseling from someone who can help them', 0.7357),
   ('that they can benefit by receiving counseling', 0.7333)]},
 {'pope francis washed and kissed the feet of muslim , orthodox , hindu and catholic refugees in a moving ceremony during holy thursday

In [ ]:
max_kw_scores = {}
for kw_set in all_keywords[:1]:
  scores = {kw[1]:kw[0] for kw in kw_set}
  print(max(scores.keys()))

e


In [ ]:
kw = pd.DataFrame(all_keywords)

In [ ]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import spacy
import spacy.cli 
spacy.cli.download("en_core_web_md")
nlp = spacy.load( "en_core_web_md", exclude=['tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer'])

keyword_model = KeyBERT(model=nlp)

In [ ]:
keyword_model.extract_keywords(tedf1['text'][0], stop_words =None)

In [ ]:
Counter(preds_task1)

Counter({0: 2094})

In [ ]:
labels2file([[k] for k in preds_task1], 'task1.txt')

# Rebuild training set (Task 2)

In [ ]:
rows2 = [] # will contain par_id, label and text
for idx in range(len(trids)):  
  parid = trids.par_id[idx]
  label = trids.label[idx]
  # select row from original dataset to retrieve the `text` value
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  rows2.append({
      'par_id':parid,
      'text':text,
      'label':label
  })
  

In [ ]:
trdf2 = pd.DataFrame(rows2)

In [ ]:
trdf2

In [ ]:
trdf2.label = trdf2.label.apply(literal_eval)

# Rebuild test set (Task 2)

In [ ]:
rows2 = [] # will contain par_id, label and text
for idx in range(len(teids)):  
  parid = teids.par_id[idx]
  label = teids.label[idx]
  #print(parid)
  # select row from original dataset to access the `text` value
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  rows2.append({
      'par_id':parid,
      'text':text,
      'label':label
  })
  

In [ ]:
tedf2 = pd.DataFrame(rows2)

In [ ]:
tedf2

In [ ]:
tedf2.label = tedf2.label.apply(literal_eval)

In [ ]:
all_negs = trdf2[trdf2.label.apply(lambda x:sum(x) == 0)]
all_pos = trdf2[trdf2.label.apply(lambda x:sum(x) > 0)]

training_set2 = pd.concat([all_pos,all_negs[:round(len(all_pos)*0.5)]])

In [ ]:
training_set2

# SpanBERT baseline for Task 2

In [ ]:
task2_model_args = MultiLabelClassificationArgs(num_train_epochs=5,
                                                no_save=True, 
                                                no_cache=True, 
                                                overwrite_output_dir=True
                                                )
task2_model = MultiLabelClassificationModel("bert", 
                                            'mrm8488/spanbert-base-finetuned-tacred', 
                                            num_labels=7,
                                            args = task2_model_args, 
                                            use_cuda=cuda_available)
# train model
task2_model.train_model(training_set2[['text', 'label']])
# run predictions
preds_task2_sp, _ = task2_model.predict(tedf2.text.tolist())

In [ ]:
labels2file(preds_task2_sp, 'task2.txt')

# Evaluate and Generate submissions

In [ ]:
!cat task2.txt | head -n 10

In [ ]:
!zip submission.zip task1.txt task2.txt

In [ ]:
!mkdir ref res

In [ ]:
import os
labels2file(tedf2.label.tolist(), os.path.join('ref/', 'task2.txt'))
#labels2file(tedf2.label.apply(lambda x:[x]).tolist(), os.path.join('ref/', 'task2.txt'))

In [ ]:
labels2file(preds_task2_sp, os.path.join('res/', 'task2.txt'))

In [ ]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/evaluation.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

In [ ]:
!python3 evaluation.py . .

In [ ]:
!cat scores.txt

In [ ]:
import os
labels2file([[k] for k in tedf1.label.tolist()], os.path.join('ref/', 'task1.txt'))

In [ ]:
labels2file([[k] for k in preds_task1], 'res/task1.txt')

In [ ]:
!python3 evaluation.py . .

In [ ]:
!cat scores.txt

In [ ]:
!cp -r /content/outputs/ /content/drive/MyDrive/ColabNotebooks/

In [ ]:
!cp -r /content/res/ /content/drive/MyDrive/ColabNotebooks/outputs/

In [ ]:
!cp -r /content/ref/ /content/drive/MyDrive/ColabNotebooks/outputs/

In [ ]:
!cp -r /content/scores.txt /content/drive/MyDrive/ColabNotebooks/outputs/

In [ ]:
! cp submission.zip /content/drive/MyDrive/ColabNotebooks/outputs/

# Eval

In [ ]:
!git clone https://github.com/facebookresearch/SpanBERT.git